[1] 모듈 로딩 및 데이터 준비 <hr>

In [28]:
## ===========================================================
## 모듈 로딩
## ===========================================================
## 일반 모듈
import pandas as pd
import numpy as np

## ML관련 모듈들
from sklearn.neighbors import KNeighborsClassifier

## 성능 평가 관련 모듈
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import TargetEncoder
from sklearn.preprocessing import OrdinalEncoder

In [29]:
## [1-2] 데이터 준비
data_file = '../Data/iris.csv'

irisDF = pd.read_csv(data_file)

[2] 데이터 학습 준비 <hr>

In [30]:
## [2-1] 피쳐/타겟, 학습용/테스트용 데이터셋 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetSR = irisDF[irisDF.columns[-1]]


In [ ]:
## 학습용 / 테스트용 데이터셋 분리
x_train, x_test, y_train, y_test = train_test_split(featureDF,
                                                    targetSR, 
                                                    test_size = 0.2,
                                                    random_state=42,
                                                    stratify=targetSR)
print(f'[Train] : {x_train.shape}, {y_train.shape}')
print(f'[Test] : {x_test.shape}, {y_test.shape}')


[Train] : (120, 4), (120,)
[Test] : (30, 4), (30,)


In [32]:
## [2-2] 타겟 컬럼 수치화 : LablelEncoder
lbEncoder = LabelEncoder()

encode_y_train = lbEncoder.fit_transform(y_train)
encode_y_test = lbEncoder.fit_transform(y_test)

print(f'classes_ : {lbEncoder.classes_}')


classes_ : ['Setosa' 'Versicolor' 'Virginica']


In [33]:
## [2-3] 수치형 피쳐 컬럼 : 스케일링
from sklearn.preprocessing import RobustScaler, StandardScaler

## => 평균 0, 분산 1 스케일링 : 이상치에 민감
stdScaler = StandardScaler()
std_x_train = stdScaler.fit_transform(x_train)
std_x_test = stdScaler.fit_transform(x_test)

## => 중간/중앙값으로 스케일링 : 이상치에 강함
rbScaler = RobustScaler()
rb_x_train = rbScaler.fit_transform(x_train)
rb_x_test = rbScaler.fit_transform(x_test)



[3] 학습 <hr>

In [48]:
## [3-1] GridSearchCV기반 교차검증 진행 => 일반화 성능 체크
from sklearn.model_selection import GridSearchCV

## 필요준비 : 모델 인스턴스, 모델의 하이퍼파라미터 dict
knn = KNeighborsClassifier()
params = {'n_neighbors': [3,5,7]}

## 교차 검증 및 최적의 하이퍼파라미터 찾기 진행
search = GridSearchCV(knn,
                      param_grid=params,
                      return_train_score=True,
                )

## 학습용 데이터셋으로 교차검증 진행 => 데이터 누수가 있음(cv)
search.fit(std_x_train, encode_y_train)

## 결과 
print('cv_results_' , search.cv_results_.keys())
print('best_score_', search.best_score_)
print('best_params_', search.best_params_)


cv_results_ dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_neighbors', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'mean_train_score', 'std_train_score'])
best_score_ 0.9666666666666668
best_params_ {'n_neighbors': 5}


In [49]:
## [3-2] 최적의 하이퍼파라미터 적용 + 전체 학습데이터로 학습 진행
best_k = search.best_params_['n_neighbors']

final_model = KNeighborsClassifier(n_neighbors=best_k)
final_model.fit(std_x_train, encode_y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


[4] 다양한 분류용 성능지표기반 평가 <hr>

In [50]:
## 성능지표함수(1D_정답, 1D_예측)
pre_y = final_model.predict(std_x_test)

## 성능지표 : 정확도
acc_1 = accuracy_score(encode_y_test, pre_y)
acc_2 = final_model.score(std_x_test, encode_y_test)
acc_1, acc_2

(1.0, 1.0)

In [52]:
# --------------------------------------------
# (3) 재현율 Recall
# --------------------------------------------
recall = recall_score(encode_y_test, pre_y, average='macro')
print("✔ Recall (macro avg)        :", recall)


✔ Recall (macro avg)        : 1.0


In [53]:
# --------------------------------------------
# (4) F1 Score
# --------------------------------------------
f1 = f1_score(encode_y_test, pre_y, average='macro')
print("✔ F1 Score (macro avg)      :", f1)

✔ F1 Score (macro avg)      : 1.0


In [54]:
# --------------------------------------------
# (5) Classification Report
# --------------------------------------------
print("\n===== 📌 Classification Report =====")
print(classification_report(encode_y_test, pre_y, target_names=lbEncoder.classes_))


===== 📌 Classification Report =====
              precision    recall  f1-score   support

      Setosa       1.00      1.00      1.00        10
  Versicolor       1.00      1.00      1.00        10
   Virginica       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

